In [2]:
!pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 251.7 kB/s eta 0:00:000:01:00:01


In [ ]:
# Загружаем данные
import pandas as pd
df = pd.read_csv('Meteorite_Landings.csv')

# 1. Удаляем строки, где нет координат
df_cleaned = df.dropna(subset=['reclat', 'reclong']).copy()

# 2. Преобразуем столбец 'year' в числовой формат. 
#    errors='coerce' превратит все некорректные значения (не числа) в NaT (Not a Time), 
#    аналог NaN для дат.
df_cleaned['year'] = pd.to_numeric(df_cleaned['year'], errors='coerce')

# 3. Удаляем строки, где год не был распознан
df_cleaned.dropna(subset=['year'], inplace=True)

# 4. Преобразуем год в целый тип, чтобы убрать ".0"
df_cleaned['year'] = df_cleaned['year'].astype(int)

# 5. Фильтруем данные: оставляем только метеориты, упавшие после 1970 года
df_filtered = df_cleaned[df_cleaned['year'] > 1970].copy()
df_filtered.dropna(subset=['mass (g)'], inplace=True)

# Выведем информацию, чтобы убедиться, что все сработало
print("Данные после полной очистки и фильтрации:")
print(df_filtered.info())
import folium

# Создаем базовую карту мира. location - начальная точка, zoom_start - масштаб
m = folium.Map(location=[20, 0], zoom_start=2)

# Чтобы карта не "тормозила", возьмем для примера первые 1000 метеоритов из отфильтрованного списка
data_for_map = df_filtered.head(1000)

# Проходим по каждой строке в наших данных и добавляем маркер на карту
for index, row in data_for_map.iterrows():
    
    # 1. Определяем цвет пина в зависимости от массы
    mass = row['mass (g)']
    if mass > 100000:  # Больше 100 кг
        pin_color = 'red'
    elif mass > 10000: # Больше 10 кг
        pin_color = 'orange'
    else:
        pin_color = 'blue'
        
    # 2. Создаем текст для всплывающего окна
    popup_text = f"""
    <b>Название:</b> {row['name']}<br>
    <b>Год падения:</b> {row['year']}<br>
    <b>Масса:</b> {mass} г.<br>
    <b>Класс:</b> {row['recclass']}
    """
    
    # 3. Добавляем сам маркер на карту
    folium.Marker(
        location=[row['reclat'], row['reclong']],
        popup=folium.Popup(popup_text, max_width=200),
        icon=folium.Icon(color=pin_color, icon='fa-star', prefix='fa') # иконка звездочки
    ).add_to(m)

# 4. Сохраняем карту в HTML-файл. Его можно будет открыть в браузере.
map_filename = 'meteorite_landings_map.html'
m.save(map_filename)

print(f"Карта сохранена в файл: {map_filename}")


Данные после полной очистки и фильтрации:
<class 'pandas.core.frame.DataFrame'>
Index: 35856 entries, 3 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         35856 non-null  object 
 1   id           35856 non-null  int64  
 2   nametype     35856 non-null  object 
 3   recclass     35856 non-null  object 
 4   mass (g)     35856 non-null  float64
 5   fall         35856 non-null  object 
 6   year         35856 non-null  int64  
 7   reclat       35856 non-null  float64
 8   reclong      35856 non-null  float64
 9   GeoLocation  35856 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 3.0+ MB
None
Карта сохранена в файл: meteorite_landings_map.html
